In [ ]:
import copy
class Node:
    def __init__(self, node, g, h, parent):
        self.state = node
        self.g = g # g: path cost aka level
        self.h = h # h1: heuristic I cost or heuristic II cost (manhatten distance)
        self.f = g + h # f: total cost = path cost + heuristic cost
        self.children = []
        self.parent = parent
    
    # returns position of a number in the state
    def get_position(self, num):
        for index, row in enumerate(self.state):
            try:
                return index, row.index(num)
            except ValueError:
                pass
        return -1,-1
    
    # generates available moves for a given state
    def generate_available_moves(self):
        self.moves = []
        row_max = 2
        col_max = 2
        self.zero = self.get_position(0) #position of zero

        i = self.zero[0]
        j = self.zero[1]

        j_dec = j - 1 #left
        if(j_dec >= 0):
            self.moves.append((i, j_dec))
        i_inc = i + 1 #bottom
        if(i_inc <= row_max):
            self.moves.append((i_inc, j))
        i_dec = i - 1 #top
        if(i_dec >= 0):
            self.moves.append((i_dec, j))
        j_inc = j + 1 #right
        if(j_inc <= col_max):
            self.moves.append((i, j_inc))
    
    # Generates new state by swapping the values of the 2 provided positions with a deep copy
    def swap_positions(self, pos_1, pos_2):
        new_state = copy.deepcopy(self.state)
        i1 = pos_1[0]
        j1 = pos_1[1]

        i2 = pos_2[0]
        j2 = pos_2[1]
        temp = new_state[i1][j1]
        new_state[i1][j1] = new_state[i2][j2]
        new_state[i2][j2] = temp

        return new_state
    
class AStarSearch:
    def __init__(self):
        self.closed = []
        self.open = []

#         self.start_state = [[2,8,3], [1,6,4], [7,0,5]]
#         self.goal_state = [[1,2,3], [8,0,4], [7,6,5]]
        
#         self.start_state = [[1,2,3], [7,4,5], [6,8,0]]
#         self.goal_state = [[1,2,3], [8,6,4], [7,5,0]]

#         self.start_state = [[2,8,1], [3,4,6], [7,5,0]]
#         self.goal_state = [[3,2,1], [8,0,4], [7,5,6]]

    def read_state(self, title):
        the_state = []
        while True:
            try:
                print("\n\n",title, " state:")
                the_state = list(map(int,input("Please type in the state in the form of numbers seperated by spaces: ").strip().split()))[:9]
                if(len(the_state) != 9):
                    print("Invalid length of the state")
                else:
                    break
            except ValueError:
                print("Wrong input: Enter Start State in the form of numbers seperated by spaces!!! ")
        return [the_state[0:3],the_state[3:6],the_state[6:9]]
    
    def read_states(self):
        self.start_state = self.read_state('Start')
        self.goal_state = self.read_state('Goal')
        self.count = 0
        
    def read_h_type(self):
        while True:
            self.h_type = input("\nPlease type 'h1' for Heuristic I or 'h2' for Heuristic II :")
            if(self.h_type == 'h1' or self.h_type == 'h2'):
                break
            else:
                print("Invalid input")
        
    
    # returns position of a number in the state
    def get_position(self, num, state):
        for index, row in enumerate(state):
            try:
                return index, row.index(num)
            except ValueError:
                pass
        return -1,-1
    
    # returns manhatten distance for a single number
    def m_dist(self, current_position, final_position):
        return abs(current_position[0] - final_position[0]) + abs(current_position[1] - final_position[1])
    
    # Returns heuristic cost I
    def h1(self,current_state, final_state):
        heuristic_value = 0
        for num in range(1,9):
            current_position = self.get_position(num, current_state)
            final_position = self.get_position(num, final_state)
            if(current_position != final_position):
                heuristic_value+= 1
        return heuristic_value
    
    # Returns heuristic cost II (i.e based on manhatten distance)
    def h2(self,current_state, final_state):
        heuristic_value = 0
        for num in range(1,9):
            current_position = self.get_position(num, current_state)
            final_position = self.get_position(num, final_state)
            m_dist = self.m_dist(current_position, final_position)

            heuristic_value+= m_dist
        return heuristic_value
    
    def print_node(self, node, info = True):
        state = node
        if(info == True):
            print("g: ", node.g, " h: ", node.h, " f: ", node.f)
            state = node.state
        for row in state:
            print(row)
        print("-------x--------")
        
    def print_path(self, node):
        self.print_node(node, False)
        if(node.parent == None):
            return
        self.print_path(node.parent)
    
    def begin(self):
        self.read_states()
        self.read_h_type()
        if(self.h_type == 'h1'):
            ini_h = self.h1(self.start_state, self.goal_state)
        else:
            ini_h = self.h2(self.start_state, self.goal_state)
            
        print("Initial State: ")
        self.print_node(self.start_state, False)
        
        print("Goal State: ")
        self.print_node(self.goal_state, False)
        
        ini_state = Node(self.start_state,0,ini_h,None)
        self.open.append(ini_state)
        
        while True:
            cur = self.open[0]
            self.count+= 1
            self.print_node(cur)
            if(cur.h == 0):
                print("++++++++++++++*****************++++++++++++++")
                #self.print_path(cur)
                break
            cur.generate_available_moves()
            for move in cur.moves:
                temp_node = cur.swap_positions(move, cur.zero)
                if(self.h_type == 'h1'):
                    temp_h = self.h1(temp_node, self.goal_state)
                else:
                    temp_h = self.h2(temp_node, self.goal_state)
                cur.children.append(Node(temp_node, cur.g+1, temp_h, cur))
                #print(cur)
            for child in cur.children:
                self.open.append(child)
            self.closed.append(cur)
            del self.open[0]
            self.open.sort(key = lambda x:x.f, reverse=False)
            
            if(self.count > 100):
                print("Unable to find a solution.")
                break

aStarSearch = AStarSearch()
aStarSearch.begin()
print("Total number of nodes expanded: ", len(aStarSearch.closed))
print("Total number of nodes generated: ", len(aStarSearch.closed) + len(aStarSearch.closed))



 Start  state:


In [4]:
some_list = input().split()

8 4 4 2 2 2 2 2 2


In [7]:
type(some_list)

list

In [12]:
len(some_list)

9

In [13]:
type(some_list[0])

str

In [11]:
1 2 3 7 4 5 6 8 0
1 2 3 8 6 4 7 5 0

SyntaxError: invalid syntax (<ipython-input-11-f951b73cf688>, line 1)

In [ ]:
1 2 3 7 4 5 6 8 0
1 2 7 8 6 4 3 5 0

In [3]:
2 8 1 3 4 6 7 5 0
3 2 1 8 0 4 7 5 6

SyntaxError: invalid syntax (<ipython-input-3-554c7733978c>, line 1)

In [ ]:
8 2 1 3 4 6 7 5 0
3 8 1 2 0 4 7 5 6

In [ ]:
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

2 8 3 1 6 4 7 0 5
[1, 2, 3]
[8, 0, 4]
[7, 6, 5]
1 2 3 8 0 4 7 6 5 

## References:

https://www.geeksforgeeks.org/python-get-a-list-as-input-from-user/

In [6]:
type(nums[1])

int

In [1]:
the_state = list(map(int,input("\nPlease type in the state in the form of numbers seperated by spaces: ").strip().split()))[:9]


Please type in the state in the form of numbers seperated by spaces: 1 2 3 4 5 6 7 8 9


[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

[4, 5, 6]

[7, 8, 9]